# Check-In Agent Per Shift

In [15]:
#importing necessary libraries.
import pandas as pd
from pulp import *
import math as mt



In [16]:
# initalize and load dataset and remove unwanted rows and replace the Nan values

week = ['Mon', 'Tue', 'Wen', 'Thu', 'Fri', 'Sat','Sun']
time_pp = 2.4
df = pd.read_csv('fau_airlines_shifts.csv')
df = df.iloc[:18]
df = df.fillna(0).applymap(lambda x: 1 if x == "X" else x)

In [17]:
# calculate service level, extract pessenger and shifts details
weeks_days = df.columns[4:]

# convert column headers to an array
weeks_days = weeks_days.to_numpy()

passengers= df[weeks_days].values

shifts_columns= df.columns[1:4]
shifts = df[shifts_columns]
shifts_pd = len(shifts_columns)

shift_slots = len(df[df[shifts_columns] == 1])
service_level = int(60 / time_pp)

print("Shift Per Day: ",shifts_pd)
print("Service Level: ",service_level)
print("Total Shift Slots: ",shift_slots)

Shift Per Day:  3
Service Level:  25
Total Shift Slots:  18


In [18]:
#creating dictionary using LpVariable.dicts
num_workers_indexes = []
for day_of_week in range (0,7) :
    for shift_index in range (shifts_pd) :
        num_workers_indexes.append (f'{day_of_week}_{shift_index}')
num_workers = LpVariable.dicts ("num_workers", num_workers_indexes, lowBound=0, cat="Integer")
print(num_workers)

{'0_0': num_workers_0_0, '0_1': num_workers_0_1, '0_2': num_workers_0_2, '1_0': num_workers_1_0, '1_1': num_workers_1_1, '1_2': num_workers_1_2, '2_0': num_workers_2_0, '2_1': num_workers_2_1, '2_2': num_workers_2_2, '3_0': num_workers_3_0, '3_1': num_workers_3_1, '3_2': num_workers_3_2, '4_0': num_workers_4_0, '4_1': num_workers_4_1, '4_2': num_workers_4_2, '5_0': num_workers_5_0, '5_1': num_workers_5_1, '5_2': num_workers_5_2, '6_0': num_workers_6_0, '6_1': num_workers_6_1, '6_2': num_workers_6_2}


In [19]:
#using LpProblem and minimize the number of check-in agents
prob = LpProblem("scheduling_workers", LpMinimize)
prob += lpSum(num_workers.values())
print(prob)

scheduling_workers:
MINIMIZE
1*num_workers_0_0 + 1*num_workers_0_1 + 1*num_workers_0_2 + 1*num_workers_1_0 + 1*num_workers_1_1 + 1*num_workers_1_2 + 1*num_workers_2_0 + 1*num_workers_2_1 + 1*num_workers_2_2 + 1*num_workers_3_0 + 1*num_workers_3_1 + 1*num_workers_3_2 + 1*num_workers_4_0 + 1*num_workers_4_1 + 1*num_workers_4_2 + 1*num_workers_5_0 + 1*num_workers_5_1 + 1*num_workers_5_2 + 1*num_workers_6_0 + 1*num_workers_6_1 + 1*num_workers_6_2 + 0
VARIABLES
0 <= num_workers_0_0 Integer
0 <= num_workers_0_1 Integer
0 <= num_workers_0_2 Integer
0 <= num_workers_1_0 Integer
0 <= num_workers_1_1 Integer
0 <= num_workers_1_2 Integer
0 <= num_workers_2_0 Integer
0 <= num_workers_2_1 Integer
0 <= num_workers_2_2 Integer
0 <= num_workers_3_0 Integer
0 <= num_workers_3_1 Integer
0 <= num_workers_3_2 Integer
0 <= num_workers_4_0 Integer
0 <= num_workers_4_1 Integer
0 <= num_workers_4_2 Integer
0 <= num_workers_5_0 Integer
0 <= num_workers_5_1 Integer
0 <= num_workers_5_2 Integer
0 <= num_workers_

In [20]:
# agent must entertain average no of pessengers
for day_of_week in range (0, 7) :
    for t in range(shift_slots) :
        prob += lpSum(
            [shifts.iloc[t, j] * num_workers[f'{day_of_week}_{j}'] 
             for j in range(shifts_pd)]) >= mt.ceil(passengers[t][day_of_week] / service_level)
        
print(prob)

scheduling_workers:
MINIMIZE
1*num_workers_0_0 + 1*num_workers_0_1 + 1*num_workers_0_2 + 1*num_workers_1_0 + 1*num_workers_1_1 + 1*num_workers_1_2 + 1*num_workers_2_0 + 1*num_workers_2_1 + 1*num_workers_2_2 + 1*num_workers_3_0 + 1*num_workers_3_1 + 1*num_workers_3_2 + 1*num_workers_4_0 + 1*num_workers_4_1 + 1*num_workers_4_2 + 1*num_workers_5_0 + 1*num_workers_5_1 + 1*num_workers_5_2 + 1*num_workers_6_0 + 1*num_workers_6_1 + 1*num_workers_6_2 + 0
SUBJECT TO
_C1: num_workers_0_0 >= 5

_C2: num_workers_0_0 >= 10

_C3: num_workers_0_0 >= 2

_C4: num_workers_0_0 >= 4

_C5: num_workers_0_0 >= 5

_C6: num_workers_0_0 >= 3

_C7: num_workers_0_1 >= 1

_C8: num_workers_0_1 >= 3

_C9: num_workers_0_1 >= 1

_C10: num_workers_0_1 >= 1

_C11: num_workers_0_1 >= 1

_C12: num_workers_0_1 >= 5

_C13: num_workers_0_2 >= 11

_C14: num_workers_0_2 >= 8

_C15: num_workers_0_2 >= 1

_C16: num_workers_0_2 >= 1

_C17: num_workers_0_2 >= 1

_C18: num_workers_0_2 >= 0

_C19: num_workers_1_0 >= 1

_C20: num_wor

In [21]:
#probe.solve is used to solve LP problem
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


In [22]:
#formulating results
for index in num_workers:
    index_parts = index.split('_')
    day_of_week = int(index_parts[0])
    day = week[day_of_week]
    shift = index_parts[1]
    print (f"The number of Agents needed for day {day} shift {shift} is {int(num_workers[index]. value())} workers")

The number of Agents needed for day Mon shift 0 is 10 workers
The number of Agents needed for day Mon shift 1 is 5 workers
The number of Agents needed for day Mon shift 2 is 11 workers
The number of Agents needed for day Tue shift 0 is 1 workers
The number of Agents needed for day Tue shift 1 is 4 workers
The number of Agents needed for day Tue shift 2 is 6 workers
The number of Agents needed for day Wen shift 0 is 2 workers
The number of Agents needed for day Wen shift 1 is 2 workers
The number of Agents needed for day Wen shift 2 is 5 workers
The number of Agents needed for day Thu shift 0 is 11 workers
The number of Agents needed for day Thu shift 1 is 11 workers
The number of Agents needed for day Thu shift 2 is 8 workers
The number of Agents needed for day Fri shift 0 is 12 workers
The number of Agents needed for day Fri shift 1 is 11 workers
The number of Agents needed for day Fri shift 2 is 3 workers
The number of Agents needed for day Sat shift 0 is 10 workers
The number of Age

# Task: 02

In [23]:
#task 02
# make changes in dataset

df['Avg_Passenger_No_3'] = df['Avg_Passenger_No_1']
df['Avg_Passenger_No_7'] = df['Avg_Passenger_No_6']

passengers *= 2
service_level = 32


print("Shift Per Day: ",shifts_pd)
print("Service Level: ",service_level)
print("Total Shift Slots: ",shift_slots)

Shift Per Day:  3
Service Level:  32
Total Shift Slots:  18


In [24]:
#creating dictionary using LpVariable.dicts
num_workers_indexes = []
for day_of_week in range (0,7) :
    for shift_index in range (shifts_pd) :
        num_workers_indexes.append (f'{day_of_week}_{shift_index}')
num_workers = LpVariable.dicts ("num_workers", num_workers_indexes, lowBound=0, cat="Integer")
print(num_workers)

{'0_0': num_workers_0_0, '0_1': num_workers_0_1, '0_2': num_workers_0_2, '1_0': num_workers_1_0, '1_1': num_workers_1_1, '1_2': num_workers_1_2, '2_0': num_workers_2_0, '2_1': num_workers_2_1, '2_2': num_workers_2_2, '3_0': num_workers_3_0, '3_1': num_workers_3_1, '3_2': num_workers_3_2, '4_0': num_workers_4_0, '4_1': num_workers_4_1, '4_2': num_workers_4_2, '5_0': num_workers_5_0, '5_1': num_workers_5_1, '5_2': num_workers_5_2, '6_0': num_workers_6_0, '6_1': num_workers_6_1, '6_2': num_workers_6_2}


In [25]:
prob = LpProblem("scheduling_workers", LpMinimize)
prob += lpSum(num_workers.values())
print(prob)

scheduling_workers:
MINIMIZE
1*num_workers_0_0 + 1*num_workers_0_1 + 1*num_workers_0_2 + 1*num_workers_1_0 + 1*num_workers_1_1 + 1*num_workers_1_2 + 1*num_workers_2_0 + 1*num_workers_2_1 + 1*num_workers_2_2 + 1*num_workers_3_0 + 1*num_workers_3_1 + 1*num_workers_3_2 + 1*num_workers_4_0 + 1*num_workers_4_1 + 1*num_workers_4_2 + 1*num_workers_5_0 + 1*num_workers_5_1 + 1*num_workers_5_2 + 1*num_workers_6_0 + 1*num_workers_6_1 + 1*num_workers_6_2 + 0
VARIABLES
0 <= num_workers_0_0 Integer
0 <= num_workers_0_1 Integer
0 <= num_workers_0_2 Integer
0 <= num_workers_1_0 Integer
0 <= num_workers_1_1 Integer
0 <= num_workers_1_2 Integer
0 <= num_workers_2_0 Integer
0 <= num_workers_2_1 Integer
0 <= num_workers_2_2 Integer
0 <= num_workers_3_0 Integer
0 <= num_workers_3_1 Integer
0 <= num_workers_3_2 Integer
0 <= num_workers_4_0 Integer
0 <= num_workers_4_1 Integer
0 <= num_workers_4_2 Integer
0 <= num_workers_5_0 Integer
0 <= num_workers_5_1 Integer
0 <= num_workers_5_2 Integer
0 <= num_workers_

In [26]:
# agent must entertain maximum no of pessengers
for day_of_week in range (0, 7) :
    for t in range(shift_slots) :
        prob += lpSum(
            [shifts.iloc[t, j] * num_workers[f'{day_of_week}_{j}'] 
             for j in range(shifts_pd)]) >= mt.ceil(passengers[t][day_of_week] / service_level)
        
print(prob)

scheduling_workers:
MINIMIZE
1*num_workers_0_0 + 1*num_workers_0_1 + 1*num_workers_0_2 + 1*num_workers_1_0 + 1*num_workers_1_1 + 1*num_workers_1_2 + 1*num_workers_2_0 + 1*num_workers_2_1 + 1*num_workers_2_2 + 1*num_workers_3_0 + 1*num_workers_3_1 + 1*num_workers_3_2 + 1*num_workers_4_0 + 1*num_workers_4_1 + 1*num_workers_4_2 + 1*num_workers_5_0 + 1*num_workers_5_1 + 1*num_workers_5_2 + 1*num_workers_6_0 + 1*num_workers_6_1 + 1*num_workers_6_2 + 0
SUBJECT TO
_C1: num_workers_0_0 >= 7

_C2: num_workers_0_0 >= 16

_C3: num_workers_0_0 >= 3

_C4: num_workers_0_0 >= 5

_C5: num_workers_0_0 >= 8

_C6: num_workers_0_0 >= 4

_C7: num_workers_0_1 >= 2

_C8: num_workers_0_1 >= 5

_C9: num_workers_0_1 >= 1

_C10: num_workers_0_1 >= 2

_C11: num_workers_0_1 >= 2

_C12: num_workers_0_1 >= 8

_C13: num_workers_0_2 >= 17

_C14: num_workers_0_2 >= 12

_C15: num_workers_0_2 >= 2

_C16: num_workers_0_2 >= 1

_C17: num_workers_0_2 >= 1

_C18: num_workers_0_2 >= 0

_C19: num_workers_1_0 >= 1

_C20: num_wo

In [27]:
#solve is used to solve LP problem
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


In [28]:
#formulating results
for index in num_workers:
    index_parts = index.split('_')
    day_of_week = int(index_parts[0])
    day = week[day_of_week]
    shift = index_parts[1]
    print (f"The number of Agents needed for day {day} shift {shift} is {int(num_workers[index]. value())} workers")

The number of Agents needed for day Mon shift 0 is 16 workers
The number of Agents needed for day Mon shift 1 is 8 workers
The number of Agents needed for day Mon shift 2 is 17 workers
The number of Agents needed for day Tue shift 0 is 1 workers
The number of Agents needed for day Tue shift 1 is 5 workers
The number of Agents needed for day Tue shift 2 is 8 workers
The number of Agents needed for day Wen shift 0 is 2 workers
The number of Agents needed for day Wen shift 1 is 3 workers
The number of Agents needed for day Wen shift 2 is 8 workers
The number of Agents needed for day Thu shift 0 is 17 workers
The number of Agents needed for day Thu shift 1 is 17 workers
The number of Agents needed for day Thu shift 2 is 12 workers
The number of Agents needed for day Fri shift 0 is 18 workers
The number of Agents needed for day Fri shift 1 is 17 workers
The number of Agents needed for day Fri shift 2 is 4 workers
The number of Agents needed for day Sat shift 0 is 15 workers
The number of Ag